# Training Abstract2Title

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
# import wandb
from datasets import load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, \
                         DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\E\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# # replace with your weights and biases username otherwise comment this
# wandb.init(project="abstract-to-title", entity="nerdimite")

## Preprocess Data

In [4]:
# Initialize T5-base tokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-base')

Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 1.38MB/s]
Downloading: 100%|██████████| 773k/773k [00:19<00:00, 40.7kB/s] 
Downloading: 100%|██████████| 1.32M/1.32M [00:39<00:00, 34.9kB/s]


In [6]:
# Load the processed data
dataset = load_from_disk('../dataset/arxiv_AI_dataset')

In [7]:
MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 128

In [8]:
def preprocess_data(example):
    
    model_inputs = tokenizer(example['abstract'], max_length=MAX_SOURCE_LEN, padding=True, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['title'], max_length=MAX_TARGET_LEN, padding=True, truncation=True)

    # Replace all pad token ids in the labels by -100 to ignore padding in the loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs['labels'] = labels["input_ids"]

    return model_inputs

In [9]:
# Apply preprocess_data() to the whole dataset
processed_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=['abstract', 'title'],
    desc="Running tokenizer on dataset",
)

processed_dataset

Running tokenizer on dataset: 100%|██████████| 3/3 [00:00<00:00,  6.15ba/s]


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 36074
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2005
    })
    val: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2004
    })
})

## Training Parameters

In [10]:
batch_size = 8
num_epochs = 5
learning_rate = 5.6e-5
weight_decay = 0.01
log_every = 50
eval_every = 1000
lr_scheduler_type = "linear"

In [11]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="model-t5-base",
    evaluation_strategy="steps",
    eval_steps=eval_every,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_steps=500,
    save_total_limit=3,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_steps=log_every,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb",
    resume_from_checkpoint=True,
)

## Train

In [12]:
# Initialize T5-base model
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

Downloading: 100%|██████████| 850M/850M [00:13<00:00, 66.9MB/s] 


In [14]:
# Define ROGUE metrics on evaluation data
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores and get the median scores
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return {k: round(v, 4) for k, v in result.items()}

In [15]:
# Dynamic padding in batch using a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
# Define the trainer
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [17]:
%%time
trainer.train()

***** Running training *****
  Num examples = 36074
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 22550
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lianaling (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 50/22550 [00:27<2:36:32,  2.40it/s]

{'loss': 2.5518, 'learning_rate': 5.5875831485587586e-05, 'epoch': 0.01}


  0%|          | 100/22550 [00:51<3:05:51,  2.01it/s]

{'loss': 2.2225, 'learning_rate': 5.5751662971175166e-05, 'epoch': 0.02}


...

100%|█████████▉| 22500/22550 [3:40:25<00:25,  2.00it/s]Saving model checkpoint to model-t5-base\checkpoint-22500
Configuration saved in model-t5-base\checkpoint-22500\config.json


{'loss': 1.4022, 'learning_rate': 1.2416851441241684e-07, 'epoch': 4.99}


Model weights saved in model-t5-base\checkpoint-22500\pytorch_model.bin
tokenizer config file saved in model-t5-base\checkpoint-22500\tokenizer_config.json
Special tokens file saved in model-t5-base\checkpoint-22500\special_tokens_map.json
Copy vocab file to model-t5-base\checkpoint-22500\spiece.model
Deleting older checkpoint [model-t5-base\checkpoint-21000] due to args.save_total_limit
100%|██████████| 22550/22550 [3:40:50<00:00,  2.89it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 22550/22550 [3:40:50<00:00,  1.70it/s]

{'loss': 1.4761, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 13259.9791, 'train_samples_per_second': 13.603, 'train_steps_per_second': 1.701, 'train_loss': 1.62230531476818, 'epoch': 5.0}
Wall time: 3h 41min 13s


TrainOutput(global_step=22550, training_loss=1.62230531476818, metrics={'train_runtime': 13259.9791, 'train_samples_per_second': 13.603, 'train_steps_per_second': 1.701, 'train_loss': 1.62230531476818, 'epoch': 5.0})

## Evaluate

In [19]:
model = AutoModelForSeq2SeqLM.from_pretrained('model-t5-base/checkpoint-22500/').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('model-t5-base/checkpoint-22500/')

loading configuration file model-t5-base/checkpoint-22500/config.json
Model config T5Config {
  "_name_or_path": "model-t5-base/checkpoint-22500/",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams":

In [20]:
%%time
trainer.evaluate(eval_dataset=processed_dataset['test'])

***** Running Evaluation *****
  Num examples = 2005
  Batch size = 8
100%|██████████| 251/251 [02:21<00:00,  1.78it/s]

Wall time: 2min 21s


{'eval_loss': 1.6426174640655518,
 'eval_rouge1': 47.9784,
 'eval_rouge2': 27.3502,
 'eval_rougeL': 42.4659,
 'eval_rougeLsum': 42.5108,
 'eval_runtime': 141.6856,
 'eval_samples_per_second': 14.151,
 'eval_steps_per_second': 1.772,
 'epoch': 5.0}

## Predict

In [21]:
temperature = 0.9
num_beams = 4
max_gen_length = 128

In [22]:
abstract = """In this paper, we question if self-supervised learning provides
new properties to Vision Transformer (ViT) [19] that
stand out compared to convolutional networks (convnets).
Beyond the fact that adapting self-supervised methods to this
architecture works particularly well, we make the following
observations: first, self-supervised ViT features contain
explicit information about the semantic segmentation of an
image, which does not emerge as clearly with supervised
ViTs, nor with convnets. Second, these features are also excellent
k-NN classifiers, reaching 78.3% top-1 on ImageNet
with a small ViT. Our study also underlines the importance of
momentum encoder [33], multi-crop training [10], and the
use of small patches with ViTs. We implement our findings
into a simple self-supervised method, called DINO, which
we interpret as a form of self-distillation with no labels.
We show the synergy between DINO and ViTs by achieving
80.1% top-1 on ImageNet in linear evaluation with ViT-Base"""
# abstract = dataset['test'][0]['abstract']
inputs = tokenizer([abstract], max_length=512, return_tensors='pt')

title_ids = model.generate(
    inputs['input_ids'].to('cuda'), 
    num_beams=num_beams, 
    temperature=temperature, 
    max_length=max_gen_length, 
    early_stopping=True
)
title = tokenizer.decode(title_ids[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(title)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Self-Supervised Vision Transformers


In [26]:
abstract = " Reception of AI paintings was positive when human subjects were unaware of the painter’s identity [24, 26] When people were told certain paintings were created by AI (attributed artist identity = AI), they rated the paintings significantly lower than other people who thought the artist was human . Therefore, we want to replicate the experiment and find out how attributed artist identity might affect the judgement on AI art ."

inputs = tokenizer([abstract], max_length=512, return_tensors='pt')

title_ids = model.generate(
    inputs['input_ids'].to('cuda'), 
    num_beams=num_beams, 
    temperature=temperature, 
    max_length=max_gen_length, 
    early_stopping=True
)
title = tokenizer.decode(title_ids[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(title)

Identifying AI Artists: The Impact of Attributed Artist Identity
